https://github.com/sharebook-kr/pykrx

In [2]:
from pykrx import stock
import pandas as pd
import dart_fss as dart

In [6]:
#공용변수
startDate = '20220708'
endDate = '20220709'

In [ ]:
#기준일,종목명,티커,시총,시총백분위

In [29]:
#마켓 : 코스닥, 코스피, 코넥스
#최근 영업일 부터 구해야 함, krx lib에서 영업일 자료가 필요함
import datetime
stock.get_previous_business_days(year=datetime.datetime.today().year
,month=datetime.datetime.today().month)[-1]
stock.get_nearest_business_day_in_a_week()

'20220708'

In [14]:
#최근 영업일 상장된 종목리스트 가져오기(코스닥, 코스피만)
tickers = stock.get_market_ticker_list(market='KOSPI') + stock.get_market_ticker_list(market='KOSDAQ')
companyName = [stock.get_market_ticker_name(ticker) for ticker in tickers ]
dfTickers = pd.DataFrame({"티커":tickers,"name":companyName})
dfTickers


,티커,name
0,095570,AJ네트웍스
1,006840,AK홀딩스
2,027410,BGF
3,282330,BGF리테일
4,138930,BNK금융지주
...,...,...
2503,024060,흥구석유
2504,010240,흥국
2505,189980,흥국에프엔비
2506,037440,희림


In [10]:
#시가총액 구하기
dfCap = stock.get_market_cap_by_ticker(startDate)

In [11]:
dfCap = dfCap.reset_index()
dfCap

,티커,종가,시가총액,거래량,거래대금,상장주식수
0,005930,58700,350426235685000,15339271,902198939736,5969782550
1,373220,388000,90792000000000,397140,152494371000,234000000
2,000660,94800,69014624202000,2814478,269081541900,728002365
3,207940,819000,58291506000000,48501,39635849000,71174000
4,005935,53900,44353593130000,790298,42485789200,822886700
...,...,...,...,...,...,...
2626,215050,912,1934959392,0,0,2121666
2627,267060,181,1654666524,0,0,9141804
2628,001529,14750,1323399500,95242,1560210800,89722
2629,329020,1485,1240200720,186,244025,835152


In [56]:
dfMain = pd.merge(dfTickers, dfCap[['티커','시가총액']],on='티커')

In [57]:
dfMain['시총순위'] = dfMain['시가총액'].rank()

In [58]:
dfMain.sort_values("시총순위")

,티커,name,시가총액,시총순위
528,009275,신원우,1134250000,1.0
311,001529,동양3우B,1323399500,2.0
68,016385,KG스틸우,2135400000,3.0
938,000547,흥국화재2우B,3486720000,4.0
1537,032685,소프트센우,3764281950,5.0
...,...,...,...,...
430,005935,삼성전자우,44353593130000,2504.0
423,207940,삼성바이오로직스,58291506000000,2505.0
151,000660,SK하이닉스,69014624202000,2506.0
85,373220,LG에너지솔루션,90792000000000,2507.0


In [59]:
dfMain['시총백분위'] = dfMain['시총순위']/dfMain.shape[0]*100

In [32]:
dfMain.query('시총백분위 <= 20').sort_values('시총백분위')

,티커,name,시가총액,시총순위,시총백분위
528,009275,신원우,1134250000,1.0,0.039872
311,001529,동양3우B,1323399500,2.0,0.079745
68,016385,KG스틸우,2135400000,3.0,0.119617
938,000547,흥국화재2우B,3486720000,4.0,0.159490
1537,032685,소프트센우,3764281950,5.0,0.199362
...,...,...,...,...,...
2088,147830,제룡산업,59800000000,497.0,19.816587
2438,054040,한국컴퓨터,59865555250,498.0,19.856459
1268,317120,라닉스,59892000000,499.0,19.896332
954,083660,CSA 코스믹,60163069325,500.0,19.936204


In [54]:
dfFundamental = stock.get_market_fundamental_by_ticker(startDate, market='ALL').reset_index('티커')[['티커','PER','PBR']]

In [60]:
dfMain = pd.merge(dfMain,dfFundamental,on='티커')
dfMain

,티커,name,시가총액,시총순위,시총백분위,PER,PBR
0,095570,AJ네트웍스,305281363400,1882.0,75.039872,3.82,0.81
1,006840,AK홀딩스,215272866250,1639.0,65.350877,0.00,0.35
2,027410,BGF,399139018470,2011.0,80.183413,6.10,0.25
3,282330,BGF리테일,3275300187000,2416.0,96.331738,22.17,4.04
4,138930,BNK금융지주,2160950680980,2374.0,94.657097,2.83,0.23
...,...,...,...,...,...,...,...
2455,024060,흥구석유,92550000000,899.0,35.845295,64.27,1.14
2456,010240,흥국,85026602400,807.0,32.177033,6.72,0.93
2457,189980,흥국에프엔비,116393513610,1145.0,45.653907,17.74,1.45
2458,037440,희림,112772047500,1125.0,44.856459,16.53,1.70


In [61]:
dfMain['1/PER'] = 1/dfMain['PER']
dfMain

,티커,name,시가총액,시총순위,시총백분위,PER,PBR,1/PER
0,095570,AJ네트웍스,305281363400,1882.0,75.039872,3.82,0.81,0.261780
1,006840,AK홀딩스,215272866250,1639.0,65.350877,0.00,0.35,inf
2,027410,BGF,399139018470,2011.0,80.183413,6.10,0.25,0.163934
3,282330,BGF리테일,3275300187000,2416.0,96.331738,22.17,4.04,0.045106
4,138930,BNK금융지주,2160950680980,2374.0,94.657097,2.83,0.23,0.353357
...,...,...,...,...,...,...,...,...
2455,024060,흥구석유,92550000000,899.0,35.845295,64.27,1.14,0.015559
2456,010240,흥국,85026602400,807.0,32.177033,6.72,0.93,0.148810
2457,189980,흥국에프엔비,116393513610,1145.0,45.653907,17.74,1.45,0.056370
2458,037440,희림,112772047500,1125.0,44.856459,16.53,1.70,0.060496


In [62]:
dfMain['1/PBR'] = 1/dfMain['PBR']
dfMain

,티커,name,시가총액,시총순위,시총백분위,PER,PBR,1/PER,1/PBR
0,095570,AJ네트웍스,305281363400,1882.0,75.039872,3.82,0.81,0.261780,1.234568
1,006840,AK홀딩스,215272866250,1639.0,65.350877,0.00,0.35,inf,2.857143
2,027410,BGF,399139018470,2011.0,80.183413,6.10,0.25,0.163934,4.000000
3,282330,BGF리테일,3275300187000,2416.0,96.331738,22.17,4.04,0.045106,0.247525
4,138930,BNK금융지주,2160950680980,2374.0,94.657097,2.83,0.23,0.353357,4.347826
...,...,...,...,...,...,...,...,...,...
2455,024060,흥구석유,92550000000,899.0,35.845295,64.27,1.14,0.015559,0.877193
2456,010240,흥국,85026602400,807.0,32.177033,6.72,0.93,0.148810,1.075269
2457,189980,흥국에프엔비,116393513610,1145.0,45.653907,17.74,1.45,0.056370,0.689655
2458,037440,희림,112772047500,1125.0,44.856459,16.53,1.70,0.060496,0.588235


In [66]:
stock.get_market_ohlcv_by_ticker(stock.get_nearest_business_day_in_a_week(), market="ALL")

,시가,고가,저가,종가,거래량,거래대금,등락률
티커,,,,,,,
060310,2955,2980,2845,2885,1087639,3166930910,-1.37
095570,6500,6580,6470,6520,45932,298342290,0.46
006840,16100,16300,16100,16250,5267,85340750,1.56
054620,8280,8280,8130,8190,27632,226210050,0.12
265520,17250,17350,16850,16900,64521,1098715600,-1.17
...,...,...,...,...,...,...,...
000547,22850,23000,22550,22700,450,10269000,2.02
000545,7000,7100,7000,7070,172,1213950,1.14
003280,2085,2140,2070,2095,245413,516984725,1.70


In [ ]:
#모멘텀 넣기, 분기, 반기, 1년